# Moby Eye Tracking

Notebook for developing fast, accurate eye tracking straight from your webcam.

In [2]:
# Imports
import os
import face_recognition
import cv2
import random
import time
import sys

import matplotlib.pyplot as plt 
import numpy as np

from tkinter import *

from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

import keras 
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, Dense, MaxPool2D, Flatten 

%matplotlib inline 

Using TensorFlow backend.


In [7]:
# Convenience functions
def small_dot(tkinter_canvas, centre_x, centre_y, radius=5, fill="red"):
    """Given the centre point of a dot, this convenience function will draw a small dot with given radius"""
    
    tkinter_canvas.create_oval(centre_x - radius, centre_y - radius,
                               centre_x + radius, centre_y + radius, fill=fill)
    
    return

def random_dot(tkinter_canvas, tk_width, tk_height):
    
    border = 5 # Should be same, or higher than radius of dots
    
    random_width = random.randint(border, tk_width - border)
    random_height = random.randint(border, tk_height - border)
    
    small_dot(tkinter_canvas, random_width, random_height)
    
    return random_width, random_height

def neural_model(dummy_sample):
    
    print("About to initialise a neural network with input shape: ", dummy_sample.shape)
    
    visible = Input(shape=(dummy_sample.shape))
    
    c11 = Conv2D(8, 3)(visible)
    c12 = Conv2D(8, 3)(c11)
    p1 = Conv2D(16, 1, strides=2)(c12)
    c21 = Conv2D(16, 3)(p1)
    c22 = Conv2D(16, 3)(c21)
    p2 = Conv2D(32, 1, strides=2)(c22)
    #c31 = Conv2D(8, 3)(p2)
    #c32 = Conv2D(8, 3)(c31)
    #p3 = Conv2D(16, 1, strides=2)(c32)
    
    f1 = Flatten()(p2)
    d1 = Dense(50, activation="relu")(f1)
    d2 = Dense(50, activation="relu")(d1)
    output = Dense(2)(d2)
    
    model = Model(inputs=visible, outputs=output)
    
    model.compile(loss=keras.losses.MeanSquaredError(), optimizer="adam")
    
    return model

def extract_facial_features(frame, display=False):
    
    # Basic code for facial landmark extraction from webcam from:
    # https://elbruno.com/2019/05/29/vscode-lets-do-some-facerecognition-with-20-lines-in-python-3-n/    
    rgb_frame = frame[:, :, ::-1].copy()
    frame_copy = frame.copy()
    bw_frame = np.mean(rgb_frame, axis=2)

    face_landmarks_list = face_recognition.face_landmarks(rgb_frame)
    
    # Extract region around eyes, before green lines added. Uses face_recognition
    border_height = 10
    border_width = 15
    
    # Creat linear ingredients to bundle with the eye data
    grad_x = np.zeros(frame_copy.shape[:2], dtype=np.float)
    grad_y = np.zeros(frame_copy.shape[:2], dtype=np.float)
    
    for i in range(border_height * 2):
        grad_x[i, :] = i / (border_height * 2)
        
    for j in range(border_width * 2):
        grad_y[:, j] = j / (border_width * 2)
    
    try:
        left_eye = np.mean(np.array(face_landmarks_list[0]["left_eye"]), axis=0, dtype=int)
        left_eye_region = bw_frame[left_eye[1] - border_height: left_eye[1] + border_height,
                                   left_eye[0] - border_width: left_eye[0] + border_width]
        left_eye_x_grad = grad_x[left_eye[1] - border_height: left_eye[1] + border_height,
                                 left_eye[0] - border_width: left_eye[0] + border_width]
        left_eye_y_grad = grad_y[left_eye[1] - border_height: left_eye[1] + border_height,
                                 left_eye[0] - border_width: left_eye[0] + border_width]
        
        left_eye_flattened = left_eye_region.reshape(1,-1)[0]
    
        right_eye = np.mean(np.array(face_landmarks_list[0]["right_eye"]), axis=0, dtype=int)
        right_eye_region = bw_frame[right_eye[1] - border_height: right_eye[1] + border_height,
                                    right_eye[0] - border_width: right_eye[0] + border_width]
        right_eye_x_grad = grad_x[right_eye[1] - border_height: right_eye[1] + border_height,
                                  right_eye[0] - border_width: right_eye[0] + border_width]
        right_eye_y_grad = grad_y[right_eye[1] - border_height: right_eye[1] + border_height,
                                  right_eye[0] - border_width: right_eye[0] + border_width]
        
        right_eye_flattened = right_eye_region.reshape(1,-1)[0]
            
        # Scale features
        scaler = StandardScaler()
        left_eye_region = scaler.fit_transform(left_eye_region)
        right_eye_region = scaler.fit_transform(right_eye_region)
        
        eyes_and_gradients = np.stack((left_eye_region, left_eye_x_grad, left_eye_y_grad,
                                       right_eye_region, right_eye_x_grad, right_eye_y_grad), axis=2)
    except IndexError:
        print("Could not extract eye regions, probably because face not detected")
        return [], [], [], []
        
    for face_landmarks in face_landmarks_list:

        for facial_feature in face_landmarks.keys():
            pts = np.array([face_landmarks[facial_feature]], np.int32) 
            pts = pts.reshape((-1,1,2))
            cv2.polylines(frame, [pts], False, (0,255,0))

    if display:
        cv2.imshow('Video', frame)
        
    # print(face_landmarks_list)
    
    # I suspect this code will break if multiple faces
    landmark_array = np.array(np.zeros((0, 2)))
    if face_landmarks_list != []:
        for landmark in face_landmarks_list[0].values():
            landmark_array = np.concatenate((landmark_array, np.array(landmark)))
    else:
        print("No face detected") 
    
    # Concatenate the extracted facial features, with the region around the eyes 
    everything_array = np.concatenate(
        (landmark_array[0], left_eye_flattened, right_eye_flattened))
    landmark_array = landmark_array[0]
    
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break
    
    everything_array = everything_array.reshape(1, -1)
    landmark_array = landmark_array.reshape(1, -1)
    
    # print(landmark_array[0].shape)
    
    return rgb_frame, everything_array, landmark_array, eyes_and_gradients

def predict_gaze(video_capture, webcam_resolution,  
                 tk_width, tk_height, model, model_type, canvas):
    
    ret, frame = video_capture.read()
    (rgb_frame, everything_array, 
     landmark_array, eyes_and_gradients) = extract_facial_features(frame)
    
    try:
        if model_type == "neural net":
            X = np.expand_dims(eyes_and_gradients, 0)
            predicted_gaze = model.predict(X)[0]
        else:
            predicted_gaze = model.predict(everything_array)[0]
    
        print("Predicted gaze is: ", predicted_gaze)
    except ValueError:
        print("Could not predict, probably no face in image")
        predicted_gaze = np.array([0., 0.])
    
    # Scale the prediction to webcam resolution
    predicted_pixel = [predicted_gaze[0] * tk_width, predicted_gaze[1] * tk_height]
    # print(predicted_pixel, predicted_gaze, webcam_resolution)
    
    # Display the prediction as a grey circle
    small_dot(canvas, predicted_pixel[0], predicted_pixel[1], radius=5, fill="grey")
    
    return rgb_frame, everything_array, eyes_and_gradients, predicted_gaze

def capture(counter, canvas, model, model_type, training_X, training_y, tk_width, tk_height, 
            video_capture, rgb_frame, webcam_resolution, 
            landmark_array, eyes_and_gradients, current_target, predicted_gaze, move_smoothly=False, randomise_dot=True):
    """Will capture an image, coordinate pair when the user is looking at the dot"""
    
    path = "data/MZeina_3/"
    train_every = 1
        
    # print("About to learn...")
    if len(landmark_array) != 0:
        current_target = np.array(current_target) / np.array([tk_width, tk_height])
        
        if model_type == "neural net":
            # Neural network can train on each sample at a time, unlike random forest
            training_X = np.expand_dims(eyes_and_gradients, 0)
            training_y = np.expand_dims(current_target, 0)
            # training_X.append(eyes_and_gradients)
        else:
            training_X.append(landmark_array[0])
            training_y.append(current_target)
        
        plt.imsave(path + str(current_target) + ".jpg", rgb_frame)
        
        if counter % train_every == 0:
            model.fit(training_X, training_y)
        
    else:
        print("Face not detected, will not train on this sample")
    
    #canvas.delete("all")
    if move_smoothly:
        speed = 20
        scaled_counter = (counter * speed) % (tk_width * tk_height)
        target_x = (scaled_counter // tk_height * speed) % tk_width
        if (scaled_counter // tk_height)%2 == 0:
            target_y = scaled_counter % tk_height
        else:
            # reverse the direction for alternative lines, so it doesn't skip up to the top
            target_y = tk_height - scaled_counter % tk_height
        print("counter, scaled_counter, are :", counter, scaled_counter)
        print("about to move small circle to", target_x, target_y)
        small_dot(canvas, target_x, target_y)
        current_target = [target_x, target_y]
    elif randomise_dot:
        current_target = random_dot(canvas, tk_width, tk_height)
    # print(random_width, random_height)
    
    return model, current_target

def train_retrospectively(path_to_images, model):
    
    # Build data frame of past images, and the extract features
    # For any non-small neural network, I should replace this technique with a generator
    
    training_X = []
    training_y = []
    counter = 0
    path_to_images = "captures_one/"
    
    # Currently only looks in a single directory
    files = os.listdir(path_to_images)
    
    for file in files:
        print("About to process image number ", counter)
        image = cv2.imread(path_to_images + file)
        rgb_frame, everything_array, landmark_array, eyes_and_gradients = extract_facial_features(image)
        coordinates = [float(coordinate) for coordinate in file[1: -5].split(" ") if len(coordinate) != 0]
        
        training_X.append(eyes_and_gradients)
        training_y.append(coordinates)
        
        counter += 1
                       
    return training_X, training_y

In [8]:
# Functions that leverage the above to do something useful
def train_and_preview(pretrained_model=None):
    ########## Universal Initialisation ##########
    counter = 0
    captures_per_point = 5
    
    ########## Initialise Video Stream ##########
    video_capture = cv2.VideoCapture(0)
    
    # Extract webcam resolution
    ret, frame = video_capture.read()
    webcam_resolution = frame.shape[:2]
    # print(webcam_resolution) 
    
    ########## Initialise ML Model ##########
    
    # Dummy sample, to help initialising models
    (rgb_frame, dummy_features, 
     landmark_array, eyes_and_gradients) = extract_facial_features(frame)
    
    model_type = "neural net"
    
    if pretrained_model:
        model = pretrained_model
    elif model_type == "random forest":
        # Random forest 
        RF = RandomForestRegressor(n_estimators=500, n_jobs=-1, warm_start=False)
        model = MultiOutputRegressor(RF)
        model.fit(np.zeros_like(dummy_features), np.array([0.5, 0.5]).reshape(1, -1))
    elif model_type == "neural net":
        model = neural_model(eyes_and_gradients)
        model.summary()
        
    # To do:Train on existing pictures
    
    # Initialise
    training_X = []
    training_y = []
    
    ########## Initialise Tkinter ##########
    window = Tk()
    window.attributes("-fullscreen", True)
    
    window.update_idletasks() 
    tk_width = window.winfo_width() 
    tk_height = window.winfo_height()

    canvas = Canvas(window, width = tk_width, height = tk_height)
    canvas.pack()
    
    window.bind("<F11>", lambda event: window.attributes("-fullscreen",
                                        not window.attributes("-fullscreen")))
    window.bind("<Escape>", lambda event: window.attributes("-fullscreen", False))
    # window.bind("c", lambda event: capture(canvas, RFMO, tk_width, tk_height, video_capture, webcam_resolution, landmark_array, current_target, predicted_gaze))
    
    # Variables to store red dot target
    current_target = random_dot(canvas, tk_width, tk_height)
    
    while True:
        
        rgb_frame, landmark_array, eyes_and_gradients, predicted_gaze = predict_gaze(
            video_capture, webcam_resolution, tk_width, tk_height, model, model_type, canvas)
        
        if counter % 4 == 0 and counter != 0:
            canvas.delete("all")
            
            RFMO, current_target = capture(
                counter, canvas, model, model_type, training_X, training_y, tk_width, tk_height, video_capture, 
                rgb_frame, webcam_resolution, landmark_array, eyes_and_gradients, 
                current_target, predicted_gaze, randomise_dot=True)
                
        counter += 1
        
        # Update GUI
        window.update_idletasks()
        window.update()
    return

In [15]:
train_and_preview(model)

Predicted gaze is:  [0.2387014  0.63727576]
Predicted gaze is:  [0.12976535 0.6904612 ]
Predicted gaze is:  [0.22098169 0.6613375 ]
Predicted gaze is:  [0.6063945  0.57704335]
Predicted gaze is:  [0.5710098  0.59300107]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0588
Predicted gaze is:  [0.7086975  0.43129587]
Predicted gaze is:  [0.7194049 0.4547298]
Predicted gaze is:  [0.70057446 0.31772578]
Predicted gaze is:  [0.7073645  0.28231424]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0169
Predicted gaze is:  [0.66527694 0.23571002]
Predicted gaze is:  [0.29426542 0.38119754]
Predicted gaze is:  [0.32782084 0.4599268 ]
Predicted gaze is:  [0.33349344 0.27198243]
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.0558
Predicted gaze is:  [0.23546815 0.31808424]
Predicted gaze is:  [0.2946688  0.18595387]
Predicted gaze is:  [0.24586411 0.27439004]
Predicted gaze is:  [0.49845502 0.23635457]
Epoch 1/1
1/1 [==========

Predicted gaze is:  [0.9043692  0.50276506]
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.0187
Predicted gaze is:  [0.7647106 0.4559178]
Predicted gaze is:  [0.93160003 0.46413517]
Predicted gaze is:  [0.82526284 0.38912958]
Predicted gaze is:  [0.94601285 0.51568246]
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.0014
Predicted gaze is:  [0.971968   0.61949027]
Predicted gaze is:  [0.6940639  0.40186235]
Predicted gaze is:  [0.6411257  0.29494527]
Predicted gaze is:  [0.7263565  0.35921878]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0272
Predicted gaze is:  [0.8103415  0.47146475]
Predicted gaze is:  [0.10234263 0.34822845]
Predicted gaze is:  [0.28057194 0.35277292]
Predicted gaze is:  [0.20834893 0.31931445]
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.0042
Predicted gaze is:  [0.17249127 0.3775415 ]
Predicted gaze is:  [0.33073738 0.7207493 ]
Predicted gaze is:  [0.5508338 0.81096

Predicted gaze is:  [0.17582324 0.6669607 ]
Predicted gaze is:  [0.20478456 0.70779014]
Predicted gaze is:  [0.19313654 0.71878135]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0073
Predicted gaze is:  [0.19797951 0.65226585]
Predicted gaze is:  [0.1939332 0.6935689]
Predicted gaze is:  [0.20717505 0.64065164]
Predicted gaze is:  [0.25091603 0.6711855 ]
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.0120
Predicted gaze is:  [0.23236705 0.707035  ]
Predicted gaze is:  [0.17765467 0.70608234]
Predicted gaze is:  [0.16831873 0.6530836 ]
Predicted gaze is:  [0.25133258 0.5931762 ]
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.0145
Predicted gaze is:  [0.2882318  0.64846575]
Predicted gaze is:  [0.2020516 0.7398636]
Predicted gaze is:  [0.21877415 0.68404126]
Predicted gaze is:  [0.21935022 0.753408  ]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0294
Predicted gaze is:  [0.21540926 0.78891

Predicted gaze is:  [0.43765086 0.14818539]
Predicted gaze is:  [0.31643227 0.29211384]
Predicted gaze is:  [0.24781056 0.37360632]
Predicted gaze is:  [0.19009219 0.3356056 ]
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.0486
Predicted gaze is:  [0.16342556 0.3113696 ]
Predicted gaze is:  [0.7176204 0.656985 ]
Predicted gaze is:  [0.6517417  0.57655966]
Predicted gaze is:  [0.7671209  0.71949464]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0161
Predicted gaze is:  [0.8175665 0.6907128]
Predicted gaze is:  [0.76925105 0.72862834]
Predicted gaze is:  [0.74422   0.6843404]
Predicted gaze is:  [0.7437196  0.62303275]
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 9.9634e-05
Predicted gaze is:  [0.8102943 0.750905 ]
Predicted gaze is:  [0.3741459  0.22969507]
Predicted gaze is:  [0.40161142 0.12272742]
Predicted gaze is:  [0.39778808 0.15469085]
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.0

Predicted gaze is:  [0.08626787 0.35004073]
Predicted gaze is:  [0.40545383 0.51901454]
Predicted gaze is:  [1.0161862 1.0095602]
Predicted gaze is:  [1.0475779 1.0492882]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0065
Predicted gaze is:  [1.0539067 1.0171708]
Predicted gaze is:  [0.45256785 0.5039658 ]
Predicted gaze is:  [0.44818392 0.08113107]
Predicted gaze is:  [0.49098882 0.12853149]
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.0025
Predicted gaze is:  [0.4507362  0.12892851]
Predicted gaze is:  [0.25223243 0.1689895 ]
Predicted gaze is:  [0.37019792 0.06216552]
Predicted gaze is:  [ 0.34465182 -0.03525537]
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.0071
Predicted gaze is:  [0.23911768 0.03302991]
Predicted gaze is:  [0.18863875 0.45136645]
Predicted gaze is:  [0.12792832 0.8905334 ]
Predicted gaze is:  [0.17546195 0.988714  ]
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.0

...

Predicted gaze is:  [0.3180132 0.7693409]
Predicted gaze is:  [0.23947136 0.77342546]
Predicted gaze is:  [0.36708593 0.86794376]
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.0013
Predicted gaze is:  [0.37890282 0.837236  ]
Predicted gaze is:  [0.6040769 0.8504208]
Predicted gaze is:  [0.6361698 0.8457645]
Predicted gaze is:  [0.6460575 0.8524401]
Epoch 1/1
1/1 [==============================] - 0s 13ms/step - loss: 0.0159
Predicted gaze is:  [0.64165115 0.9235269 ]
Predicted gaze is:  [0.80345064 0.8328876 ]
Predicted gaze is:  [0.80043095 0.7388121 ]
Predicted gaze is:  [0.7842261 0.7356261]
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.0015
Predicted gaze is:  [0.68184453 0.6994476 ]
Predicted gaze is:  [ 0.5373993  -0.01360996]
Predicted gaze is:  [0.5109883  0.01930772]
Predicted gaze is:  [ 0.5480579  -0.00932765]
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.0021
Predicted gaze is:  [0.59450674 0.013195

Predicted gaze is:  [0.79126376 0.28524572]
Predicted gaze is:  [0.7113403 0.6393462]
Predicted gaze is:  [0.8010288  0.63475895]
Predicted gaze is:  [0.7358725 0.6479579]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0013
Predicted gaze is:  [0.8162167 0.6968705]
Predicted gaze is:  [0.7384379  0.35523638]
Predicted gaze is:  [0.73519564 0.3165659 ]
Predicted gaze is:  [0.7930223  0.24828699]
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.0013
Predicted gaze is:  [0.7006847  0.24945067]
Predicted gaze is:  [0.4617781 0.7751975]
Predicted gaze is:  [0.48855227 0.90741974]
Predicted gaze is:  [0.47570103 0.8501568 ]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0031
Predicted gaze is:  [0.4951663 0.7912609]
Predicted gaze is:  [0.45798677 0.47708154]
Predicted gaze is:  [0.4348976  0.36924064]
Predicted gaze is:  [0.4391631 0.4393968]
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.0021
Pred

Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Face not detected, will not train on this sample
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Face not detected, will not train on this sample
Could not extr

Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Face not detected, will not train on this sample
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Face not detected, will not train on this sample
Could not extract eye regions, probably because face not detected
Could not predict, probably no face in image
Could not extr

TclError: invalid command name ".!canvas"

In [ ]:
video_capture = cv2.VideoCapture(0)
RF = RandomForestRegressor(n_estimators=100, n_jobs=-1, warm_start=True)
RFMO = MultiOutputRegressor(RF)
RFMO.fit(np.zeros_like(extract_facial_features(video_capture)), np.array([0, 0]).reshape(1, -1))
# RFMO.predict(np.array([1,1,1]).reshape(1, -1))
# RFMO.fit(np.array([1,1,1]).reshape(1, -1), np.array([1, 0]).reshape(1, -1))

In [10]:
class ScreenshotGenerator(keras.utils.Sequence):
    
    def __init__(self, path_to_images, batch_size=4):
        
        self.path_to_images = path_to_images
        self.batch_size = batch_size
    
        self.files = []# os.listdir(path_to_images)
        self.filenames = []
        
        for root, dirs, files in os.walk(path_to_images):
            for name in files:
                self.files.append(os.path.join(root, name))
                self.filenames.append(name)
    
    def __len__(self):
        
        return len(self.files) // self.batch_size
    
    def __load__(self, index):
        """Returns and processes a single sample, in conjunction with __getitem__"""
        
        # Ensures that if an image is picked without a succesfully detected face, 
        #  it looks for another random one to replace it
        got_good_image = False
        
        while not got_good_image:
        
            file = self.files[index]
            filename = self.filenames[index]
                        
            image = cv2.imread(file)
            
            rgb_frame, everything_array, landmark_array, eyes_and_gradients = extract_facial_features(image)
            coordinates = [float(coordinate) for coordinate in filename[1: -5].split(" ") if len(coordinate) != 0]
            
            X = eyes_and_gradients
            y = coordinates
            
            if len(X) == 0:
                print("This image did not have a recognisable face, will pull a random one in its place")
                index = random.randint(0, self.__len__())
            else:
                got_good_image = True
        
        return X, y
    
    def __getitem__(self, batch):
        
        batch_X = [self.__load__(index)[0] for index in 
                   range((batch * self.batch_size), (batch + 1) * self.batch_size)]
        batch_y = [self.__load__(index)[1] for index in 
                   range((batch * self.batch_size), (batch + 1) * self.batch_size)]
        
        batch_X = np.array(batch_X)
        batch_y = np.array(batch_y)
        
        return batch_X, batch_y

    def on_epoch_end(self):
        
        return

In [11]:
video_capture = cv2.VideoCapture(0)
_, frame = video_capture.read()
video_capture.release()

In [12]:
(rgb_frame, dummy_features, 
     landmark_array, eyes_and_gradients) = extract_facial_features(frame)

In [13]:
model = neural_model(eyes_and_gradients)
#model.summary()

About to initialise a neural network with input shape:  (20, 30, 6)


In [14]:
screenshot_generator = ScreenshotGenerator("data/", 4)
model.fit_generator(screenshot_generator, epochs=4)

Epoch 1/4
273/858 [========>.....................] - ETA: 24:15 - loss: 0.0422

KeyboardInterrupt: 